<a href="https://colab.research.google.com/github/playtipus69/python/blob/main/telegram_chatbot_with_ia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Instalação de Bibliotecas


In [53]:
!pip install pytelegrambotapi
!pip install unidecode

# Lendo nosso dataset

## Pandas


A biblioteca pandas é uma poderosa biblioteca em Python amplamente utilizada para análise de dados e manipulação de estruturas de dados tabulares, como DataFrames. Ela oferece funcionalidades para carregar, limpar, transformar, analisar e visualizar dados de maneira eficiente.

In [54]:
import pandas as pd

In [55]:
dataset = pd.read_csv('https://raw.githubusercontent.com/tmelo-uea/cursos/main/dataset.csv', delimiter=';')

In [56]:
dataset.head(n=99)

,Pergunta,Resposta
0,Bom dia!,Bom dia! Como posso ajudar você hoje?
1,Boa tarde!,Boa tarde! Estou à disposição para responder à...
2,Boa noite!,Boa noite! Em que posso ser útil?
3,Oi!,Olá! Como posso ser útil a você?
4,"Olá, viajante!",Olá! Estou aqui para fornecer informações sobr...
...,...,...
94,Onde posso fazer rapel na Chapada dos Veadeiro...,A Chapada dos Veadeiros oferece oportunidades ...
95,Quais são as atrações de ecoturismo em Jalapão...,Jalapão é perfeito para ecoturismo. Faça trilh...
96,"O que fazer em São Miguel dos Milagres, Alagoa...",São Miguel dos Milagres é um refúgio tranquilo...
97,"Quais são as atrações culturais em São Luís, M...","São Luís tem um centro histórico preservado, c..."


# Bibliotecas para tratar strings



In [57]:
import string

# Bibliotecas de NLP

## NLTK
O Natural Language Toolkit (NLTK) é uma biblioteca em Python amplamente utilizada para processamento de linguagem natural (NLP). Ela oferece uma gama de ferramentas que tornam mais fácil a realização de análises textuais e tarefas de NLP, bem como a construção de sistemas de processamento de linguagem natural.

In [58]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

# Biblioteca para remoção de acentuação

In [59]:
from unidecode import unidecode

# Pré processamento de texto

In [60]:
def remove_pontuacao(text):
    texto_limpo = ''
    for palavra in text:
      if palavra not in string.punctuation:
        texto_limpo += palavra

    return texto_limpo

In [61]:
print(string.punctuation)

!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~


In [62]:
def preprocessamento(texto):
    # Remove pontuações e símbolos
    texto = remove_pontuacao(texto)

    # Remover acentos
    texto = unidecode(texto)

    # Converte para minúsculo
    texto = texto.lower()

    # Tokenização
    tokens = word_tokenize(texto)

    # Remover stopwords
    stop_words = stopwords.words('portuguese')
    tokens = [token for token in tokens if token not in stop_words]

    # Juntar as palavras novamente em uma string
    texto_preprocessado = ' '.join(tokens)

    return texto_preprocessado

In [63]:
dataset["Pergunta_Preprocessada"] = dataset["Pergunta"].apply(preprocessamento)

In [64]:
dataset.head(n=1000)

,Pergunta,Resposta,Pergunta_Preprocessada
0,Bom dia!,Bom dia! Como posso ajudar você hoje?,bom dia
1,Boa tarde!,Boa tarde! Estou à disposição para responder à...,boa tarde
2,Boa noite!,Boa noite! Em que posso ser útil?,boa noite
3,Oi!,Olá! Como posso ser útil a você?,oi
4,"Olá, viajante!",Olá! Estou aqui para fornecer informações sobr...,ola viajante
...,...,...,...
314,Como é a culinária típica do estado da Bahia?,A culinária da Bahia é conhecida por seus sabo...,culinaria tipica estado bahia
315,O que fazer na região dos Lençóis Maranhenses?,Os Lençóis Maranhenses são famosos por suas du...,fazer regiao lencois maranhenses
316,Quais são as atrações naturais imperdíveis no ...,"Além dos Lençóis Maranhenses, visite o Monte R...",quais sao atracoes naturais imperdiveis brasil
317,Como é o transporte entre cidades no Brasil?,Você pode viajar entre cidades brasileiras de ...,transporte cidades brasil


## Vetorização

In [65]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [66]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(dataset["Pergunta_Preprocessada"])

## Método para obter uma resposta a partir de uma pergunta

In [67]:
def obter_resposta(pergunta):
    # Pré processa a sentença da pergunta
    pergunta_processada = preprocessamento(pergunta)

    # Transforma a pergunta para um vetor
    pergunta_vector = vectorizer.transform([pergunta_processada])

    # Calcula a similaridade do cosseno
    similaridades = cosine_similarity(pergunta_vector, tfidf_matrix)

    # Obtém o índice da pergunta mais similar
    pergunta_index = similaridades.argmax()

    # Devolve a resposta para o usuário
    return dataset["Resposta"].iloc[pergunta_index]

## Testando Chatbot

In [68]:
flags = ['Fechar', 'Sair', 'Tchau']
end = False

while not end:
    question = input()
    if question in flags:
        end = True
        print('Finalizando Chat')
        continue

    answer = obter_resposta(question)
    print(answer)

sair
Bom dia! Como posso ajudar você hoje?
sair
Bom dia! Como posso ajudar você hoje?
Sair
Finalizando Chat


# Integrando com Telegram

In [69]:
import telebot

API_KEY = '7550433956:AAFAu3HWeVSh-BX3vvg_c7o5oZckpAHuZiU'
bot = telebot.TeleBot(API_KEY)

In [70]:
@bot.message_handler(func=lambda message: True)
def default(message):
    resposta = obter_resposta(message.text)

    bot.reply_to(
        message,
        resposta
    )

In [71]:
bot.polling()